In [11]:
from geopy.geocoders import Nominatim

# Fonction pour obtenir les coordonnées d'une adresse
def get_coordinates(address):
    geolocator = Nominatim(user_agent="sise_o_resto", timeout=10)
    current_address = address
    while True:
        location = geolocator.geocode(f"{current_address}, Rhône, France")
        if location:
            return location.latitude, location.longitude
        if ',' in current_address:
            before_comma, after_comma = current_address.split(',', 1)
            before_words = before_comma.strip().split(' ')
            if len(before_words) > 1:
                # Supprimer le dernier mot avant la virgule
                before_words = before_words[:-1]
                new_before = ' '.join(before_words)
                current_address = f"{new_before}, {after_comma.strip()}"
            else:
                # Si un seul mot avant la virgule, le supprimer complètement
                current_address = after_comma.strip()
        else:
            break
    return None, None

In [12]:
get_coordinates("20 Rue Sergent-Blandan Parking Sathonay, 69001 Lyon France")

(45.7686963, 4.8293587)

In [87]:
import urllib.parse
from geopy.geocoders import Nominatim

def get_coordinates(address):
    geolocator = Nominatim(user_agent="streamlit_app", timeout=10)
    location = geolocator.geocode(f"{address}, Rhône, France")
    if location:
        return location.latitude, location.longitude

departure_address = "Rue de la république"
arrival_address = "5 Av. Pierre Mendès France, 69500 Bron"
    
dep_lat, dep_lon = get_coordinates(departure_address)
arr_lat, arr_lon = get_coordinates(arrival_address)
            
if dep_lat and dep_lon and arr_lat and arr_lon:
    from_coord = f"{dep_lon};{dep_lat}"
    to_coord = f"{arr_lon};{arr_lat}"
    encoded_from = urllib.parse.quote(from_coord)
    encoded_to = urllib.parse.quote(to_coord)
    
    tcl_url = f"https://www.tcl.fr/itineraires?date=now&pmr=0&from={encoded_from}&to={encoded_to}"
    print(tcl_url)

https://www.tcl.fr/itineraires?date=now&pmr=0&from=4.83559115%3B45.76019985&to=4.9151265%3B45.7225931


In [89]:
import requests

# URL de l'API
tcl_api_url = f"https://carte.tcl.fr/api/itinerary?datetime=now&from={encoded_from}&to={encoded_to}&params=departure,metro,funiculaire,tramway,bus,shuttle,bss,bike,car&walking=1.12&cycling=4.44"

# Headers pour simuler un navigateur
headers = {
    "Accept": "application/json, text/plain, */*",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7",
    "Referer": f"https://carte.tcl.fr/route-calculation?date=now&from={encoded_from}&to={encoded_to}&lang=fr",
    "Sec-Ch-Ua": '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": '"Windows"',
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-origin",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
    "DNT": "1",  # Do Not Track
}

# Effectuer la requête GET à l'API
response = requests.get(tcl_api_url, headers=headers)

# Vérifier si la requête a réussi
if response.status_code == 200:
    # Convertir la réponse en JSON
    data = response.json()
    
    # Extraire la durée depuis la première entrée des trajets
    if "journeys" in data and len(data["journeys"]) > 0:
        duration = data["journeys"][0]["duration"]
        # Conversion de la durée en minutes et si plus de 60 minutes, en heures
        duration = duration // 60
        if duration > 60:
            hours = duration // 60
            minutes = duration % 60
            duration = f"{hours}h{minutes}"
        else:
            duration = f"{duration} min"
        print(f"Durée du trajet : {duration}")

Durée du trajet : 38 min
